In [1]:
from glob import glob
import pandas as pd
from sklearn.model_selection import GroupKFold
import cv2
from skimage import io
import albumentations as A
import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.nn import functional as F
from glob import glob
import sklearn
from warmup_scheduler import GradualWarmupScheduler
from torch import nn
import warnings

warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=DeprecationWarning) 

SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)


In [2]:
warnings.simplefilter('ignore')
torch.manual_seed(47)
np.random.seed(47)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
device_ids=range(torch.cuda.device_count())


In [3]:
device = torch.device("cuda:0")

In [4]:
DATA_PATH = '/home/xhzhu/data/my_siim'
df_folds = pd.read_csv(f'{DATA_PATH}/folds_13062020.csv', index_col='image_id')
set(df_folds[df_folds['fold'] == 0]['patient_id'].values).intersection(df_folds[df_folds['fold'] == 1]['patient_id'].values)

set()

In [5]:
df_folds

,patient_id,target,source,sex,age_approx,anatom_site_general_challenge,stratify_group,fold
image_id,,,,,,,,
ISIC_2637011,IP_7279968,0,ISIC20,male,45.0,head/neck,31,0
ISIC_0015719,IP_3075186,0,ISIC20,female,45.0,upper extremity,7,2
ISIC_0052212,IP_2842074,0,ISIC20,female,50.0,lower extremity,5,4
ISIC_0068279,IP_6890425,0,ISIC20,female,45.0,head/neck,7,0
ISIC_0074268,IP_8723313,0,ISIC20,female,55.0,upper extremity,6,4
...,...,...,...,...,...,...,...,...
ISIC_0071181,BCN_0002740,0,ISIC19,female,30.0,lower extremity,0,1
ISIC_0071317,BCN_0003003,0,ISIC19,male,80.0,lower extremity,19,4
ISIC_0071505,BCN_0005307,0,ISIC19,female,40.0,torso,0,0


In [6]:
test_df = pd.read_csv("/home/xhzhu/data/siim/test.csv", index_col='image_name')
test_df

,patient_id,sex,age_approx,anatom_site_general_challenge
image_name,,,,
ISIC_0052060,IP_3579794,male,70.0,NaN
ISIC_0052349,IP_7782715,male,40.0,lower extremity
ISIC_0058510,IP_7960270,female,55.0,torso
ISIC_0073313,IP_6375035,female,50.0,torso
ISIC_0073502,IP_0589375,female,45.0,lower extremity
...,...,...,...,...
ISIC_9992485,IP_4152479,male,40.0,torso
ISIC_9996992,IP_4890115,male,35.0,torso
ISIC_9997917,IP_2852390,male,25.0,upper extremity


In [7]:

# One-hot encoding of anatom_site_general_challenge feature
concat = pd.concat([df_folds['anatom_site_general_challenge'], test_df['anatom_site_general_challenge']], ignore_index=False)
dummies = pd.get_dummies(concat, dummy_na=True, dtype=np.uint8, prefix='site')
df_folds = pd.concat([df_folds.loc[:, :], dummies.iloc[:df_folds.shape[0]]], axis=1)
test_df = pd.concat([test_df, dummies.iloc[df_folds.shape[0]:].reset_index(drop=True)], axis=1)

# Sex features
df_folds['sex'] = df_folds['sex'].map({'male': 1, 'female': 0})
test_df['sex'] = test_df['sex'].map({'male': 1, 'female': 0})
df_folds['sex'] = df_folds['sex'].fillna(-1)
test_df['sex'] = test_df['sex'].fillna(-1)

# Age features
df_folds['age_approx'] /= df_folds['age_approx'].max()
test_df['age_approx'] /= test_df['age_approx'].max()
#train_df['age_approx'] = train_df['age_approx'].fillna(0)
test_df['age_approx'] = test_df['age_approx'].fillna(0)

df_folds['patient_id'] = df_folds['patient_id'].fillna(0)
print(df_folds)

               patient_id  target  source  sex  age_approx  \
image_id                                                     
ISIC_2637011   IP_7279968       0  ISIC20  1.0    0.500000   
ISIC_0015719   IP_3075186       0  ISIC20  0.0    0.500000   
ISIC_0052212   IP_2842074       0  ISIC20  0.0    0.555556   
ISIC_0068279   IP_6890425       0  ISIC20  0.0    0.500000   
ISIC_0074268   IP_8723313       0  ISIC20  0.0    0.611111   
...                   ...     ...     ...  ...         ...   
ISIC_0071181  BCN_0002740       0  ISIC19  0.0    0.333333   
ISIC_0071317  BCN_0003003       0  ISIC19  1.0    0.888889   
ISIC_0071505  BCN_0005307       0  ISIC19  0.0    0.444444   
ISIC_0071594  BCN_0004562       1  ISIC19  0.0    0.555556   
ISIC_0072850  BCN_0001347       0  ISIC19  0.0    0.555556   

             anatom_site_general_challenge  stratify_group  fold  \
image_id                                                           
ISIC_2637011                     head/neck              3

In [8]:
meta_features = ['sex', 'age_approx'] + [col for col in df_folds.columns if 'site_' in col]
meta_features.remove('anatom_site_general_challenge')
print(meta_features)

['sex', 'age_approx', 'site_head/neck', 'site_lateral torso', 'site_lower extremity', 'site_oral/genital', 'site_palms/soles', 'site_torso', 'site_unknown', 'site_upper extremity', 'site_nan']


In [9]:
print(df_folds.head(10))

              patient_id  target  source  sex  age_approx  \
image_id                                                    
ISIC_2637011  IP_7279968       0  ISIC20  1.0    0.500000   
ISIC_0015719  IP_3075186       0  ISIC20  0.0    0.500000   
ISIC_0052212  IP_2842074       0  ISIC20  0.0    0.555556   
ISIC_0068279  IP_6890425       0  ISIC20  0.0    0.500000   
ISIC_0074268  IP_8723313       0  ISIC20  0.0    0.611111   
ISIC_0074311  IP_2950485       0  ISIC20  0.0    0.444444   
ISIC_0074542  IP_4698288       0  ISIC20  1.0    0.277778   
ISIC_0075663  IP_6017204       0  ISIC20  0.0    0.388889   
ISIC_0075914  IP_7622888       0  ISIC20  1.0    0.333333   
ISIC_0076262  IP_5075533       0  ISIC20  0.0    0.555556   

             anatom_site_general_challenge  stratify_group  fold  \
image_id                                                           
ISIC_2637011                     head/neck              31     0   
ISIC_0015719               upper extremity               7     

In [10]:
df_folds[df_folds['fold'] == 0]

,patient_id,target,source,sex,age_approx,anatom_site_general_challenge,stratify_group,fold,site_head/neck,site_lateral torso,site_lower extremity,site_oral/genital,site_palms/soles,site_torso,site_unknown,site_upper extremity,site_nan
image_id,,,,,,,,,,,,,,,,,
ISIC_2637011,IP_7279968,0,ISIC20,1.0,0.500000,head/neck,31,0,1,0,0,0,0,0,0,0,0
ISIC_0068279,IP_6890425,0,ISIC20,0.0,0.500000,head/neck,7,0,1,0,0,0,0,0,0,0,0
ISIC_0074542,IP_4698288,0,ISIC20,1.0,0.277778,lower extremity,27,0,0,0,1,0,0,0,0,0,0
ISIC_0076262,IP_5075533,0,ISIC20,0.0,0.555556,lower extremity,7,0,0,0,1,0,0,0,0,0,0
ISIC_0076995,IP_2235340,0,ISIC20,0.0,0.611111,torso,5,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ISIC_0069229,BCN_0000402,0,ISIC19,1.0,0.722222,torso,19,0,0,0,0,0,0,1,0,0,0
ISIC_0069769,BCN_0001158,0,ISIC19,0.0,0.944444,head/neck,0,0,1,0,0,0,0,0,0,0,0
ISIC_0070306,BCN_0002580,0,ISIC19,0.0,0.333333,palms/soles,0,0,0,0,0,0,1,0,0,0,0


In [11]:

def get_train_transforms():
    return A.Compose([
            A.RandomSizedCrop(min_max_height=(400, 400), height=512, width=512, p=0.5),
            A.RandomRotate90(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
#             A.RandomBrightness(p=0.5),
#             A.RandomContrast(p=0.5),
#             A.RandomGamma(p=0.5),
#             A.ChannelShuffle(p=0.5),
            #A.Resize(height=512, width=512, p=1),
            A.Cutout(num_holes=64, max_h_size=32, max_w_size=32, fill_value=0, p=0.5),
            ToTensorV2(p=1.0),                  
        ], p=1.0)

def get_valid_transforms():
    return A.Compose([
            A.RandomRotate90(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
#             A.RandomBrightness(p=0.5),
#             A.RandomContrast(p=0.5),
#             A.RandomGamma(p=0.5),
#             A.ChannelShuffle(p=0.5),
            #A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)


In [12]:
TRAIN_ROOT_PATH = f'{DATA_PATH}/512x512-dataset-melanoma/512x512-dataset-melanoma/'

def onehot(size, target):
    vec = torch.zeros(size, dtype=torch.float32)
    vec[target] = 1.
    return vec

class DatasetRetriever(Dataset):

    def __init__(self, image_ids, labels, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, idx: int):
        image_id = self.image_ids[idx]
        image = cv2.imread(f'{TRAIN_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = image.astype(np.float32) / 255.0

        label = self.labels[idx]

        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']

        target = onehot(2, label)
        return image, target

    def __len__(self) -> int:
        return self.image_ids.shape[0]

    def get_labels(self):
        return list(self.labels)


In [13]:
class MelanomaDataset(Dataset):
    def __init__(self, df: pd.DataFrame, image_ids, labels, train: bool = True, transforms = None, meta_features = None):
        """
        Class initialization
        Args:
            df (pd.DataFrame): DataFrame with data description
            imfolder (str): folder with images
            train (bool): flag of whether a training dataset is being initialized or testing one
            transforms: image transformation method to be applied
            meta_features (list): list of features with meta information, such as sex and age
            
        """
        self.df = df
        self.transforms = transforms
        self.train = train
        self.meta_features = meta_features
        self.image_ids = image_ids
        self.labels = labels
        
        
    def __getitem__(self, index):
        image_id = self.image_ids[index]
        im_path = f'{TRAIN_ROOT_PATH}/{image_id}.jpg'
        x = cv2.imread(im_path, cv2.IMREAD_COLOR)
        x = x.astype(np.float32) / 255.0
        meta = np.array(self.df.iloc[index][self.meta_features].values, dtype=np.float32)
        label = self.labels[index]

        if self.transforms:
            sample = {'image': x}
            sample = self.transforms(**sample)
            x = sample['image']
            
#         if self.train:
#             y = onehot(2, label)
#             return (x, meta), y
#         else:
#             return (x, meta)
        y = onehot(2, label)
        return (x, meta), y
    
    def __len__(self):
        return len(self.df)
    
    def get_labels(self):
        return list(self.labels)
    
    
class Net(nn.Module):
    def __init__(self, arch, n_meta_features: int):
        super(Net, self).__init__()
        self.arch = arch
#         if 'ResNet' in str(arch.__class__):
#             self.arch.fc = nn.Linear(in_features=512, out_features=500, bias=True)
#         if 'EfficientNet' in str(arch.__class__):
        self.arch._fc = nn.Linear(in_features=1536, out_features=500, bias=True)
        self.meta = nn.Sequential(nn.Linear(n_meta_features, 100),
                                  nn.BatchNorm1d(100),
                                  nn.ReLU(),
                                  nn.Dropout(p=0.2),
                                  nn.Linear(100, 50),  # FC layer output will have 250 features
                                  nn.BatchNorm1d(50),
                                  nn.ReLU(),
                                  nn.Dropout(p=0.2))
        self.ouput = nn.Linear(500 + 50, 2)
        
    def forward(self, inputs):
        """
        No sigmoid in forward because we are going to use BCEWithLogitsLoss
        Which applies sigmoid for us when calculating a loss
        """
        x = inputs[0]
        meta = inputs[1]
#         print(x.shape, "^^^^^^^^^^^^^")
#         print(meta.shape, "###########")
#         f = open("arch.txt", 'w')
#         print(arch, file=f)
#         f.close
#         print(self.arch, "**********")
#         print(x.device, "@@@@@@@@@@@@@")
        cnn_features = self.arch(x)

        meta_features = self.meta(meta)
        features = torch.cat((cnn_features, meta_features), dim=1)
        output = self.ouput(features)
        return output


In [14]:
from sklearn import metrics

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


class RocAucMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.y_true = np.array([0,1])
        self.y_pred = np.array([0.5,0.5])
        self.score = 0

    def update(self, y_true, y_pred):
        y_true = y_true.cpu().numpy().argmax(axis=1).clip(min=0, max=1).astype(int)
        # y_pred = 1 - nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:,0]
        y_pred = nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:,1]
        self.y_true = np.hstack((self.y_true, y_true))
        self.y_pred = np.hstack((self.y_pred, y_pred))
        self.score = sklearn.metrics.roc_auc_score(self.y_true, self.y_pred)

    @property
    def avg(self):
        return self.score
    
    def get_true(self):
        return self.y_true[2:]

    def get_pred(self):
        return self.y_pred[2:]


    
class APScoreMeter(RocAucMeter):
    def __init__(self):
        super(APScoreMeter, self).__init__()

    def update(self, y_true, y_pred):
        y_true = y_true.cpu().numpy().argmax(axis=1).clip(min=0, max=1).astype(int)
        y_pred = nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:,1]
        self.y_true = np.hstack((self.y_true, y_true))
        self.y_pred = np.hstack((self.y_pred, y_pred))
        self.score = sklearn.metrics.average_precision_score(self.y_true, self.y_pred)

In [15]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets)

        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss
        
class LabelSmoothing(nn.Module):
    def __init__(self, smoothing = 0.1):
        super(LabelSmoothing, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing

    def forward(self, x, target):
        if self.training:
            x = x.float()
            target = target.float()
            logprobs = torch.nn.functional.log_softmax(x, dim = -1)

            nll_loss = -logprobs * target
            nll_loss = nll_loss.sum(-1)
    
            smooth_loss = -logprobs.mean(dim=-1)

            loss = self.confidence * nll_loss + self.smoothing * smooth_loss

            return loss.mean()
        else:
            return torch.nn.functional.cross_entropy(x, target)


In [16]:
from efficientnet_pytorch import EfficientNet

def get_net():
    net = EfficientNet.from_pretrained('efficientnet-b3')
    #net._fc = nn.Linear(in_features=1536, out_features=2, bias=True)
    #._fc = nn.Linear(in_features=2304, out_features=2, bias=True)
    return net

arch = get_net()
net = Net(arch=arch, n_meta_features=len(meta_features))
net = nn.DataParallel(net, device_ids=device_ids)
net.cuda()



Loaded pretrained weights for efficientnet-b3


DataParallel(
  (module): Net(
    (arch): EfficientNet(
      (_conv_stem): Conv2dStaticSamePadding(
        3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
        (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
      )
      (_bn0): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_blocks): ModuleList(
        (0): MBConvBlock(
          (_depthwise_conv): Conv2dStaticSamePadding(
            40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
            (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
          )
          (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
          (_se_reduce): Conv2dStaticSamePadding(
            40, 10, kernel_size=(1, 1), stride=(1, 1)
            (static_padding): Identity()
          )
          (_se_expand): Conv2dStaticSamePadding(
            10, 40, kernel_size=(1, 1), stride=(1, 1)
    

In [17]:

class Fitter:
    
    def __init__(self, model, device, config, folder):
        self.config = config
        self.epoch = 0

        self.base_dir = f'model/meta_b3_new/512_16/{folder}'
        if not os.path.exists(self.base_dir):
            os.makedirs(self.base_dir)

        self.log_path = f'{self.base_dir}/log.txt'
        self.best_score = 0
        self.best_loss = 10**5
        self.best_ap = 0
        
        self.model = model
        self.device = device
        self.best_true = np.array([])
        self.best_pred = np.array([])

        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ] 

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        self.scheduler = config.SchedulerClass(self.optimizer, **config.scheduler_params)
        self.scheduler_warmup = GradualWarmupScheduler(self.optimizer, multiplier=1, total_epoch=6)

#         self.criterion = FocalLoss(logits=True).to(self.device)
        self.criterion = LabelSmoothing().to(self.device)
        self.log(f'Fitter prepared. Device is {self.device}')

    def fit(self, train_loader, validation_loader):
        for e in range(self.config.n_epochs):
            if self.config.verbose:
                lr = self.optimizer.param_groups[0]['lr']
                timestamp = datetime.utcnow().isoformat()
                self.log(f'\n{timestamp}\nLR: {lr}')
                
            if self.epoch <= 6:
                self.scheduler_warmup.step(self.epoch)
                print(self.epoch, self.optimizer.param_groups[0]['lr'])
                
            t = time.time()
            summary_loss, roc_auc_scores, ap_scores = self.train_one_epoch(train_loader)
            self.log(f'[RESULT]: Train. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, roc_auc: {roc_auc_scores.avg:.5f}, ap: {ap_scores.avg:.5f}, time: {(time.time() - t):.5f}')

            t = time.time()
            f_true, f_pred, summary_loss, roc_auc_scores, ap_scores = self.validation(validation_loader)

            self.log(f'[RESULT]: Val. Epoch: {self.epoch}, summary_loss: {summary_loss.avg:.5f}, roc_auc: {roc_auc_scores.avg:.5f}, ap: {ap_scores.avg:.5f}, time: {(time.time() - t):.5f}')
            if summary_loss.avg < self.best_loss:
                self.best_loss = summary_loss.avg
                self.save_model(f'{self.base_dir}/best-loss-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-loss-checkpoint-*epoch.bin'))[:-2]:
                    os.remove(path)
                    
            if roc_auc_scores.avg > self.best_score:
                self.best_score = roc_auc_scores.avg
                self.save_model(f'{self.base_dir}/best-score-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-score-checkpoint-*epoch.bin'))[:-2]:
                    os.remove(path)
                self.best_true = f_true
                self.best_pred = f_pred
                    
            if ap_scores.avg > self.best_ap:
                self.best_ap = ap_scores.avg
                self.save_model(f'{self.base_dir}/best-ap-checkpoint-{str(self.epoch).zfill(3)}epoch.bin')
                for path in sorted(glob(f'{self.base_dir}/best-ap-checkpoint-*epoch.bin'))[:-2]:
                    os.remove(path)

            if self.config.validation_scheduler:
                if self.epoch > 6:
                    self.scheduler.step(metrics=summary_loss.avg)

            self.epoch += 1
        return self.best_true , self.best_pred

    def validation(self, val_loader):
        self.model.eval()
        summary_loss = AverageMeter()
        roc_auc_scores = RocAucMeter()
        ap_scores = APScoreMeter()
        t = time.time()
        for step, (images, targets) in enumerate(val_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Val Step {step}/{len(val_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, roc_auc: {roc_auc_scores.avg:.5f}, ap: {ap_scores.avg:.5f} ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            with torch.no_grad():
                images[0] = torch.as_tensor(images[0], device=device, dtype=torch.float32)
                images[1] = torch.as_tensor(images[1], device=device, dtype=torch.float32)
                targets = torch.as_tensor(targets, device=device, dtype=torch.float32)
                batch_size = images[0].shape[0]
                outputs = self.model(images)
                loss = self.criterion(outputs, targets)
                roc_auc_scores.update(targets, outputs)
                ap_scores.update(targets, outputs)
                summary_loss.update(loss.detach().item(), batch_size)
        f_true = roc_auc_scores.get_true()
        f_pred = roc_auc_scores.get_pred()

        return f_true, f_pred, summary_loss, roc_auc_scores, ap_scores

    def train_one_epoch(self, train_loader):
        self.model.train()
        summary_loss = AverageMeter()
        roc_auc_scores = RocAucMeter()
        ap_scores = APScoreMeter()
        t = time.time()
        for step, (images, targets) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f'Train Step {step}/{len(train_loader)}, ' + \
                        f'summary_loss: {summary_loss.avg:.5f}, roc_auc: {roc_auc_scores.avg:.5f}, ap: {ap_scores.avg:.5f} ' + \
                        f'time: {(time.time() - t):.5f}', end='\r'
                    )
            images[0] = torch.as_tensor(images[0], device=device, dtype=torch.float32)
            images[1] = torch.as_tensor(images[1], device=device, dtype=torch.float32)
            targets = torch.as_tensor(targets, device=device, dtype=torch.float32)
#             targets = targets.to(self.device).float()
#             images = images.to(self.device).float()
#             print(images[0].shape, "&&&")
            batch_size = images[0].shape[0]
#             print(batch_size, "batch size***")

            self.optimizer.zero_grad()
#             print(np.array(images).shape, "*****************")
            outputs = self.model(images)
            loss = self.criterion(outputs, targets)
            loss.backward()
            
            roc_auc_scores.update(targets, outputs)
            ap_scores.update(targets, outputs)
            summary_loss.update(loss.detach().item(), batch_size)

            self.optimizer.step()

            if self.config.step_scheduler:
                self.scheduler.step()

        return summary_loss, roc_auc_scores, ap_scores
    
    def save_model(self, path):
        self.model.eval()
        torch.save(self.model.state_dict(),path)

    def save(self, path):
        self.model.eval()
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_score': self.best_score,
            'best_ap': self.best_ap,
            'best_loss': self.best_loss,
            'epoch': self.epoch,
        }, path)

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.best_score = checkpoint['best_score']
        self.best_ap = checkpoint['best_ap']
        self.best_loss = checkpoint['best_loss']
        self.epoch = checkpoint['epoch']
        
    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, 'a+') as logger:
            logger.write(f'{message}\n')


In [18]:
class TrainGlobalConfig:
    num_workers = 2
    batch_size = 16
    n_epochs = 30
    lr = 0.00003

    # -------------------
    verbose = True
    verbose_step = 1
    # -------------------

    # --------------------
    step_scheduler = False  # do scheduler.step after optimizer.step
    validation_scheduler = True  # do scheduler.step after validation stage loss

#     SchedulerClass = torch.optim.lr_scheduler.OneCycleLR
#     scheduler_params = dict(
#         max_lr=0.001,
#         epochs=n_epochs,
#         steps_per_epoch=int(len(train_dataset) / batch_size),
#         pct_start=0.1,
#         anneal_strategy='cos', 
#         final_div_factor=10**5
#     )
    
    SchedulerClass = torch.optim.lr_scheduler.ReduceLROnPlateau
    scheduler_params = dict(
        mode='min',
        factor=0.8,
        patience=2,
        verbose=False, 
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0, 
        min_lr=1e-8,
        eps=1e-08
    )


In [19]:

fitter = Fitter(model=net, device=torch.device('cuda:0'), config=TrainGlobalConfig, folder='base_state')
BASE_STATE_PATH = f'{fitter.base_dir}/base_state.bin'
fitter.save(BASE_STATE_PATH)

Fitter prepared. Device is cuda:0


In [20]:


from catalyst.data.sampler import BalanceClassSampler

def train_fold(fold_number):
    
#     train_dataset = DatasetRetriever(
#         image_ids=df_folds[df_folds['fold'] != fold_number].index.values,
#         labels=df_folds[df_folds['fold'] != fold_number].target.values,
#         transforms=get_train_transforms(),
#     )
    train_dataset = MelanomaDataset(df=df_folds[df_folds['fold'] != fold_number], 
                            image_ids=df_folds[df_folds['fold'] != fold_number].index.values,
                            labels=df_folds[df_folds['fold'] != fold_number].target.values,
                            train=True, 
                            transforms=get_train_transforms(),
                            meta_features=meta_features)
    df_val = df_folds[(df_folds['fold'] == fold_number) & (df_folds['source'] == 'ISIC20')]
   
    validation_dataset = MelanomaDataset(df=df_val, 
                            image_ids=df_val.index.values,
                            labels=df_val.target.values,      
                            train=False, 
                            transforms=get_valid_transforms(),
                            meta_features=meta_features)
#     validation_dataset = DatasetRetriever(
#         image_ids=df_val.index.values,
#         labels=df_val.target.values,
#         transforms=get_valid_transforms(),
#     )

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        sampler=BalanceClassSampler(labels=train_dataset.get_labels(), mode="downsampling"),
        batch_size=TrainGlobalConfig.batch_size,
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
    )
    val_loader = torch.utils.data.DataLoader(
        validation_dataset, 
        batch_size=TrainGlobalConfig.batch_size,
        num_workers=TrainGlobalConfig.num_workers,
        shuffle=False,
        sampler=SequentialSampler(validation_dataset),
        pin_memory=False,
    )

    fitter = Fitter(model=net, device=torch.device('cuda:0'), config=TrainGlobalConfig, folder=f'fold{fold_number}')
    fitter.load(BASE_STATE_PATH)
    f_true, f_pred = fitter.fit(train_loader, val_loader)
    return f_true, f_pred



In [21]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 

all_true = [np.array([]), np.array([]), np.array([]), np.array([]), np.array([])]
all_pred = [np.array([]), np.array([]), np.array([]), np.array([]), np.array([])]

for fold_number in range(5):      # range(5)
    all_true[fold_number], all_pred[fold_number] = train_fold(fold_number=fold_number)
    print(all_true[fold_number].shape, all_pred[fold_number].shape)

final_true = np.hstack((all_true[0], all_true[1], all_true[2], all_true[3], all_true[4]))
final_pred = np.hstack((all_pred[0], all_pred[1], all_pred[2], all_pred[3], all_pred[4]))
K = pd.DataFrame(columns = ["true", "pred"])
K['true'] = final_true
K['pred'] = final_pred
K.to_csv('model/meta_b3_new/512_16/oof.csv',index=False)
al_auc = sklearn.metrics.roc_auc_score(final_true, final_pred)
print("all_roc_auc is:", al_auc)
    

Fitter prepared. Device is cuda:0

2020-08-16T05:37:04.827392
LR: 0.0
0 0.0


/home/xhzhu/anaconda3/envs/py37/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning:

Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate

/home/xhzhu/anaconda3/envs/py37/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning:

The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.



[RESULT]: Train. Epoch: 0, summary_loss: 0.70475, roc_auc: 0.46863, ap: 0.48428, time: 193.61809
[RESULT]: Val. Epoch: 0, summary_loss: 0.67340, roc_auc: 0.55244, ap: 0.02075, time: 53.94857

2020-08-16T05:41:12.650671
LR: 0.0
1 4.9999999999999996e-06


/home/xhzhu/anaconda3/envs/py37/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning:

The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.



[RESULT]: Train. Epoch: 1, summary_loss: 0.62836, roc_auc: 0.74774, ap: 0.74009, time: 199.35359
[RESULT]: Val. Epoch: 1, summary_loss: 0.44410, roc_auc: 0.81365, ap: 0.09123, time: 53.51134

2020-08-16T05:45:25.748368
LR: 4.9999999999999996e-06
2 9.999999999999999e-06


...

[RESULT]: Train. Epoch: 4, summary_loss: 0.49837, roc_auc: 0.87534, ap: 0.86774, time: 200.17182
[RESULT]: Val. Epoch: 4, summary_loss: 0.30782, roc_auc: 0.87676, ap: 0.18719, time: 53.26176

2020-08-16T14:39:48.233702
LR: 1.9999999999999998e-05
5 2.5e-05


/home/xhzhu/anaconda3/envs/py37/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning:

The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.



[RESULT]: Train. Epoch: 5, summary_loss: 0.48740, roc_auc: 0.88479, ap: 0.87391, time: 198.82556
[RESULT]: Val. Epoch: 5, summary_loss: 0.28889, roc_auc: 0.88860, ap: 0.20115, time: 53.77740

2020-08-16T14:44:01.113513
LR: 2.5e-05
6 3e-05


/home/xhzhu/anaconda3/envs/py37/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:143: UserWarning:

The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.



[RESULT]: Train. Epoch: 6, summary_loss: 0.47454, roc_auc: 0.89543, ap: 0.89019, time: 197.82608
[RESULT]: Val. Epoch: 6, summary_loss: 0.30202, roc_auc: 0.89414, ap: 0.19624, time: 52.89322

2020-08-16T14:48:11.980834
LR: 3e-05
[RESULT]: Train. Epoch: 7, summary_loss: 0.46830, roc_auc: 0.90011, ap: 0.89554, time: 200.17530
[RESULT]: Val. Epoch: 7, summary_loss: 0.29990, roc_auc: 0.90314, ap: 0.20910, time: 54.08786

2020-08-16T14:52:26.422792
LR: 3e-05
[RESULT]: Train. Epoch: 8, summary_loss: 0.46280, roc_auc: 0.90382, ap: 0.89668, time: 197.20820
[RESULT]: Val. Epoch: 8, summary_loss: 0.29615, roc_auc: 0.89605, ap: 0.20631, time: 53.80418

2020-08-16T14:56:37.437794
LR: 3e-05
[RESULT]: Train. Epoch: 9, summary_loss: 0.44283, roc_auc: 0.91816, ap: 0.91527, time: 197.46144
[RESULT]: Val. Epoch: 9, summary_loss: 0.29906, roc_auc: 0.90675, ap: 0.25082, time: 53.91895

2020-08-16T15:00:49.021007
LR: 3e-05
[RESULT]: Train. Epoch: 10, summary_loss: 0.43755, roc_auc: 0.92166, ap: 0.91940, ti